In [ ]:
# ✅ Cell 1: Setup & Load Input Data (with dynamic alias)

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from utils.notebook_setup import enable_project_imports
enable_project_imports()

from utils.io_helpers import list_files_with_aliases, load_file_with_alias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, lit, concat_ws, size, desc

# 🚀 Start Spark
spark = SparkSession.builder.appName("Pyspark - Clean & Transform").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


In [ ]:
  # from google.colab import drive

In [ ]:
   # drive.mount('/content/drive')

In [ ]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
data_type ='json'
path = '../data/input/practice/'+ data_type
data_df = spark.read.format(data_type).load(path)
data_df.show()

In [ ]:
data_df.printSchema()

In [ ]:
data_df.show()

In [ ]:
data_df=spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/example/parquet/part-00000-097d5e0c-28b8-4e7f-baa7-b6228ea55df9-c000.snappy.parquet')

In [ ]:
data_df.show()

In [ ]:
data_type ='parquet'
path = '../data/input/practice/'+ data_type
data_practice = spark.read.format(data_type).load(path)
data_practice.show()

In [ ]:
from pyspark.sql import functions as f

In [ ]:
data_practice.count()

In [ ]:
data_practice.show(truncate=False)

Curățarea coloanei „nume” de spații suplimentare - Se recomandă folosirea funcției trim sau replace.

Curățarea coloanei „ocupatie” de sparții suplimentare -  Se recomandă folosirea funcției trim sau replace.

Eliminarea diacriticelor din coloana „nume” -

Scoaterea valorilor duplicate din coloana „extra”

Setarea coloanei „inactiv” pe false atunci când valoarea lipsește. - Se recomandă folosirea funcției coalesce.


In [ ]:
df_tasks = (
    data_practice
    .withColumn('nume', f.trim(f.col('nume'))   )
    .withColumn('ocupatie',f.trim(f.col('ocupatie')))
    .withColumn('inactiv',f.coalesce(f.col('inactiv'),f.lit(False) ) )
    .withColumn('nume',f.translate(f.col('nume'),'șțăîâ','staia' ))
    .withColumn('extra',f.array_distinct(f.col('extra')))
)

In [ ]:
df_tasks.show()

Adăugarea coloanei „varstă_la_contractare” egală cu diferența dintre vârsta persoanei și vechime

Scoaterea coloanei „vechime”

Adăugarea coloanei „text_descriptiv” care să fie un text ce descrie clientul și conține neapărat toate informațiile știute despre el, mai puțin despre starea contractului dacă este activ. (e.g. X, în vârstă de ... ani,
este ...). Atenție la cazul special la coloana „extra” când numărul de elemente este 0. -  Se recomandă cel folosirea funcțiilor when și printf.

In [ ]:
df_transformare= (
    data_practice
    .withColumn('varsta_la_contractare',f.col('varsta') - f.col('vechime'))
    .drop(f.col('vechime'))
  #  .withColumn('array_extra', f.array_join(f.col('extra'),', '))
  #  .withColumn('extra cnt', f.size(f.col('extra') ) )
    .withColumn('text_descriptiv',
                f.concat(
        f.col('nume'),
        f.lit(', in varsta de '),
        f.col('varsta'),
        f.lit(' ani, este '),
        f.col('ocupatie'),

        f.when(
            f.size(f.col('extra') ) > 0 ,
               f.concat(
                    f.lit(' cu extra activitati in : ') ,
                    f.array_join(f.col('extra'),', '))
        )
        .otherwise(f.lit(''))
                                                                    ))
)

In [ ]:
df_transformare.show(truncate=False)

### Afișarea clienților inactivi / Afișarea contractelor inactive

In [ ]:
df_inactive_contracts = df_tasks.filter('inactiv is True')
df_inactive_contracts.show(20)
#df_inactive_contracts.count()


### Afișarea clienților cu minim și maxim de vechime / Afișarea contractelor care au minim 2 ani de vechime și maxim 5 ani vechime, inclusiv.

In [ ]:
df_vechime_2y_to_5y = (
    df_tasks
    .filter(f.col('vechime') >= 2)
    .filter(f.col('vechime') <= 5)
)
df_vechime_2y_to_5y.show(20)

### Afișarea celor mai în vârstă persoane cu orice fel de contract  / Afișarea top 10 contracte sortate după vârsta lor descrescător

In [ ]:
df_invarsta = (
    df_tasks
    .sort(f.desc_nulls_last(f.col('varsta')))
    .limit(10)
    )

df_invarsta.show()

### Afișarea celor mai în vârstă oameni la contractare / Afișarea top 5 contracte active sortate după vârsta la contractare descrescător

In [ ]:
df_4 = (
    df_tasks
    .withColumn('varsta la contractare',(f.col('varsta') - f.col('vechime')))
)
df_4.sort(f.desc('varsta la contractare')).limit(5).show()

Reutilizarea Operațiilor

 Împărțiți setul de date în patru segmente, folosindu-vă de coloana "inactiv" și criteriul vârstei.

 Creați un segment pentru cei cu vârsta de până la 30 de ani inclusiv și un altul pentru cei peste 30 de ani, aplicând
această divizare atât pentru înregistrările active, cât și pentru cele inactive.

 Scrieți fiecare set de date într-o locație diferită.

 Înainte de scrierea fiecărui segment, afișați datele (nu uitați să cache-uiți setul de date și înainte de
afișare, nu numai înainte de împărțire)

In [ ]:

cached_data_df = df_tasks.select('varsta', 'inactiv').cache()

df_active_young = (
    cached_data_df
    .filter(f.col('varsta') <=  f.lit(30) )
     .filter('inactiv is false')
    .cache()
)


df_inactive_young = (
    cached_data_df
    .filter(f.col('varsta') <=  f.lit(30) )
    .filter('inactiv is True')
    .cache()
)

df_active_old = (
    cached_data_df
    .filter(f.col('varsta') >  f.lit(30) )
    .filter('inactiv is False')
    .cache()
)

df_inactive_old = (
    cached_data_df
    .filter(f.col('varsta') >  f.lit(30) )
    .filter('inactiv is True')
    .cache()
)




In [ ]:
df_active_young.show()

In [ ]:
df_inactive_young.show()

In [ ]:
df_active_old.show()

In [ ]:
df_inactive_old.show()

In [ ]:
df_active_young.write.format('parquet').save('/content/drive/MyDrive/Colab Notebooks/practice/outputs/1')
df_inactive_young.write.format('parquet').save('/content/drive/MyDrive/Colab Notebooks/practice/outputs/2')
df_active_old.write.format('parquet').save('/content/drive/MyDrive/Colab Notebooks/practice/outputs/3')
df_inactive_old.write.format('parquet').save('/content/drive/MyDrive/Colab Notebooks/practice/outputs/4')


In [ ]:
from pyspark.sql import functions as f

In [ ]:
data_without_personal_columns_df = data_practice.drop('varsta', 'extra')

In [ ]:
data_without_personal_columns_df.show()

In [ ]:
df_rename = data_without_personal_columns_df.withColumnRenamed('ocupatie','job')

In [ ]:
df_rename.show()

In [ ]:
df_rename.explain('extended')

In [ ]:
print((f.col('varsta') * f.lit(2) + f.col('vechime') - f.lit(3)) / f.lit(2) < 40)

In [ ]:
df_rename.printSchema()

In [ ]:
df_clean_nume = (
    data_practice
    .withColumn('nume', f.trim(f.col('nume'))   )
    .withColumn('job',f.trim(f.col('job')))
    .withColumn()

)

In [ ]:
df_clean_nume.show()